---

# **Weather LLM agent**

---

In [1]:
!pip install langchain langchain_community openai

In [ ]:

import os
import requests
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
import warnings
warnings.filterwarnings("ignore")

# Set your API keys
os.environ["OPENAI_API_KEY"] = "###########"
weather_api_key = "#########" # Replace with your weather API key

# Function to fetch weather data from OpenWeatherMap
def get_weather(city):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={weather_api_key}&units=metric"
    response = requests.get(url).json()
    if response.get("cod") != 200:
        return f"Weather data not found for {city}."
    weather = response["weather"][0]["description"]
    temp = response["main"]["temp"]
    return f"The current weather in {city} is {weather} with a temperature of {temp}°C."

# Define weather tool
weather_tool = Tool(
    name="weather",
    func=lambda city: get_weather(city),
    description="Provides current weather information for a given city."
)

# Initialize the language model (OpenAI GPT-4 or GPT-3.5)
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# Set up memory for personalized conversations
memory = ConversationBufferMemory()

# Initialize the agent with tools and memory
agent = initialize_agent(
    tools=[weather_tool],  # Add more tools if needed (e.g., flight, hotel, etc.)
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# Example user interactions
# 1. User asks for a sunny destination
user_query = input("Enter destination location : ")
response_1 = agent.run(user_query)
print(response_1)
